<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Exploración DataVAULT
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>


## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Abril 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Consideraciones para la consulta en datos abiertos**|
|**Descripción y alcance**|Script para la consulta de bases y exploración de bases en el DATAVAULT.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC<td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

![image.png](../../../documentos/analisis%20y%20descubrimientos/generacion_entidades_.png)

## Instalaciones previas
 
Para poder utilizar SQL en Python es necesario instalar lo siguiente, escoja el driver de acuerdo al sistema operativo de su equipo: 
 
 
**Driver Microsoft ODBC para SQL SERVER en windows**
 
Descargar e instalar el driver de la página [Microsoft ODBC](https://docs.microsoft.com/en-us/sql/connect/odbc/windows/system-requirements-installation-and-driver-files?view=sql-server-ver15#installing-microsoft-odbc-driver-for-sql-server) versión 
 17.
 
**Driver Microsoft ODBC para SQL SERVER en Linux o MacOs**
 
Descargar e instalar el driver de la página [Microsoft ODBC](https://docs.microsoft.com/en-us/sql/connect/odbc/linux-mac/system-requirements?view=sql-server-ver15) versión 17 deacuerdo a la versión de su sistema operativo.
Recuerde estar conectado a la VPN para poder accerder. 

**Paquete pyobdc**

Quite el comentario e inicie la instalación simple.

In [1]:
import pyodbc 
from dotenv import load_dotenv
import os
load_dotenv()

server=os.getenv("server")
username=os.getenv("username")
password=os.getenv("password")

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [2]:
### Paquetes usados para la exploración de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
from scripts import data_management as dm # Funciones definidas
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## 3. DESCRIPCIÓN DEL INFORME / INSUMO

### 3.1. Objetivo

El objetivo de este cuaderno es mostrar la información de Entidades dispuesta en la Bodega de Datos de Colombia Compra Eficiente, para que los usuarios puedan consultarla y explorarla.

## 4. ADQUISICIÓN DE LOS DATOS

### 4.1. Datos de procesos SECOP II

Para consultar los datos de procesos SECOP II, se debe ejecutar la siguiente consulta:

```sql
SELECT * FROM [CCE_Sandbox].[datavault].[SAT_Example_Table]
```


Se crea una ruta:

In [3]:
ruta_DATA='../../../Data/Raw'

In [4]:
# Se lee la consulta de entidades

DF_ENTIDADES = pd.read_sql_query("SELECT * FROM [CCE_Sandbox].[datavault].[Sat_Company]", cnxn)

In [5]:
DF_ENTIDADES.columns[DF_ENTIDADES.columns.str.contains('Date')]

Index(['LoadDate', 'CreateDate', 'EndDateContractOrder',
       'StartDateContractOrder', 'VersionDate', 'VortalTrustEndDate',
       'VortalTrustStartDate'],
      dtype='object')

In [6]:
DF_ENTIDADES=DF_ENTIDADES.sort_values(by=['CreateDate'], ascending=False)

In [7]:
# Se extraen las variables necesarias
df_entidades=DF_ENTIDADES[['Code','LegalName','VAT','IsBuyer','CreateDate']]

# Se pasa a mayúscula el nombre legal y VAT
df_entidades['LegalName']=dm.mayuscula(df_entidades,'LegalName')
df_entidades['VAT']=dm.mayuscula(df_entidades,'VAT')

# Se agrupa por Code
df_entidades_first=df_entidades.groupby('Code').agg({'LegalName':'first','VAT':'first','IsBuyer':'first','CreateDate':'first'})
df_entidades_count=df_entidades.groupby('Code').agg({'LegalName':'nunique','VAT':'nunique','IsBuyer':'nunique'})

## Integramos
df_entidades=pd.merge(df_entidades_first,df_entidades_count,how='left',on='Code')
df_entidades=df_entidades.rename(columns={'LegalName_x':'LegalName','VAT_x':'VAT','IsBuyer_x':'IsBuyer','LegalName_y':'LegalName_count','VAT_y':'VAT_count','IsBuyer_y':'IsBuyer_count'})
df_entidades=df_entidades.reset_index()

# Se filtra que Code no contenga '\d'
df_entidades_err=df_entidades[~df_entidades['Code'].str.contains('\d',regex=True)]

# Se exporta a un parquet
df_entidades_err.to_parquet(ruta_DATA+'/Entidades_erroneas.parquet',index=False)

# Se filtra que Code no contenga '_'
df_entidades=df_entidades[~df_entidades['Code'].str.contains('_')]

Guardamos los datos problematicos en la carpeta *Problemas_Sat_Company* para revisarlos después, separamos las entidades, los proveedores y las entidades. Así mismo, hay algunos que no tienen esta identificación por la variable IsBuyer, por lo que se separan en la carpeta Otros.

In [8]:
# Se filtra donde el conteo de nombre legal sea mayor a 2

Problematicos=df_entidades[df_entidades['LegalName_count']>2]
Problematicos.shape

(492, 8)

In [9]:
# Filtro de Code

DF_ENTIDADES[DF_ENTIDADES['Code']=='700058068']

,HubCompanyHashKey,LoadDate,RecordSource,SatCompanyHashDiff,AccountBankCode,AccountBankName,AccountIBAN,AccountName,AccountNumber,AccountNumberPrefix,...,VortalTrust,VortalTrustDocument,VortalTrustEndDate,VortalTrustStartDate,WantToDO,WebAddress,YouTubeUrl,Deleted,RepresentativeCertificate,CompanySize
1188364,5822BC169CED142DCE275600F5D4CABE,2022-11-04 11:33:59.407000,Community.dbo.Company,AAEA33B510C987409A6FCD8F73777AFA,None,BANCO DE BOGOTA,None,None,079283685,None,...,False,None,None,None,1.0,None,None,None,None,0.0
1129786,5822BC169CED142DCE275600F5D4CABE,2022-11-10 13:40:48.860000,Community.dbo.Company,7ECD5F75C3B2473DBA350DA902BC557E,None,BANCO DE BOGOTA,None,None,079283685,None,...,False,None,None,None,1.0,None,None,None,None,0.0
504013,5822BC169CED142DCE275600F5D4CABE,2022-03-11 19:59:21.261859,Community.dbo.Company,F49030C597C978820C52C0CFD9289E00,None,BANCO DE BOGOTA,None,None,981132624,None,...,False,None,None,None,1.0,https://alnaturalser.com/,None,None,None,NaN


Guardamos los problematicos en la ruta: `../../../Data/Processed/Problemas_Sat_Company`

En el código montamos una línea que verifica si existe la carpeta y si no la crea.

In [10]:
import os
import re
ruta = '../../../Data/Processed/Problemas_Sat_Company'
if not os.path.exists(ruta):
    os.makedirs(ruta)
    os.makedirs(ruta+'/Entidades')
    os.makedirs(ruta+'/Provedores')
    os.makedirs(ruta+'/Otros')

In [11]:
## Creación de ruta para guardar los archivos

Lista_Fechas=[]
Problematicos.index=Problematicos['Code']
for i in Problematicos.index:
    if Problematicos['IsBuyer'][i]==True:
        name_file=re.sub('[^A-Za-z0-9]','_',Problematicos['LegalName'][i])
        DF_ENTIDADES[DF_ENTIDADES['Code']==i].to_excel(ruta+'/Entidades/'+name_file+'.xlsx')
    elif Problematicos['IsBuyer'][i]==False:
        name_file=re.sub('[^A-Za-z0-9]','_',Problematicos['LegalName'][i])
        DF_ENTIDADES[DF_ENTIDADES['Code']==i].to_excel(ruta+'/Provedores/'+name_file+'.xlsx')
    else:
        name_file=re.sub('[^A-Za-z0-9]','_',Problematicos['LegalName'][i])
        DF_ENTIDADES[DF_ENTIDADES['Code']==i].to_excel(ruta+'/Otros/'+name_file+'.xlsx')


Dejamos guardada descargada la información a nivel local en la ruta:

`/dropbox/Dropbox/CCE-projects/Large Data`

(Depende del usuario)

In [12]:
ruta_LD='/dropbox/Dropbox/CCE-projects/Large Data'

In [13]:
### Modificamos formato para la columna LoadDate, cdctimestamp,CreateDate
DF_ENTIDADES['LoadDate']=pd.to_datetime(DF_ENTIDADES['LoadDate'])
DF_ENTIDADES['LoadDate']=DF_ENTIDADES['LoadDate'].dt.strftime('%Y-%m-%d')
DF_ENTIDADES['LoadDate']=DF_ENTIDADES['LoadDate'].astype(str)

DF_ENTIDADES['cdctimestamp']=pd.to_datetime(DF_ENTIDADES['cdctimestamp'])
DF_ENTIDADES['cdctimestamp']=DF_ENTIDADES['cdctimestamp'].dt.strftime('%Y-%m-%d')
DF_ENTIDADES['cdctimestamp']=DF_ENTIDADES['cdctimestamp'].astype(str)

DF_ENTIDADES['CreateDate']=pd.to_datetime(DF_ENTIDADES['CreateDate'])
DF_ENTIDADES['CreateDate']=DF_ENTIDADES['CreateDate'].dt.strftime('%Y-%m-%d')
DF_ENTIDADES['CreateDate']=DF_ENTIDADES['CreateDate'].astype(str)

In [14]:
df_entidades.columns

Index(['Code', 'LegalName', 'VAT', 'IsBuyer', 'CreateDate', 'LegalName_count',
       'VAT_count', 'IsBuyer_count'],
      dtype='object')

In [15]:
## Guardamos los archivos como parquet usando pyarrow
df_entidades.columns=['Codigo', 'Nombre Legal', 'NIT', 'Entidad','Creacion_Fecha','Count Nombre Legal', 'Count NIT', 'Count Entidad']
df_entidades.to_parquet(ruta_LD+'/Entidades/Sat_Company.parquet',engine='pyarrow')
DF_ENTIDADES.to_parquet(ruta_LD+'/Entidades/Sat_Company_full.parquet',engine='pyarrow')

In [16]:
# Filtro de entidades y exportación a un parquet

DF_ENTIDADES=DF_ENTIDADES[DF_ENTIDADES['IsBuyer']==True]
DF_ENTIDADES.to_parquet(ruta_LD+'/Entidades/Entidades_SECOP_II_FULL.parquet',engine='pyarrow')

df_entidades=df_entidades[df_entidades['Entidad']==True]
df_entidades.to_parquet(ruta_LD+'/Entidades/Entidades_SECOP_II.parquet',engine='pyarrow')

## Carga de datos al Datalake

Cargamos la base desde la ruta DATA




In [17]:
import pandas as pd
import pyarrow.parquet as pq
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# URL del punto final del servicio Blob y token SAS
url_blob = "https://adlindicadoresemae.blob.core.windows.net/"
token_sas = "?sv=2021-10-04&ss=btqf&srt=sco&st=2023-04-26T10%3A55%3A03Z&se=2023-07-27T10%3A55%3A00Z&sp=rwdxftlacup&sig=9%2FBvXPQRTnl4CUwGzrUpMFXbpu50nNRqix5pREicSiY%3D"

# Conexión a la cuenta de almacenamiento de Azure utilizando el token SAS
connect_str = f"BlobEndpoint={url_blob};SharedAccessSignature={token_sas}"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [18]:
# Referencia al contenedor donde se guardará el archivo
container_name_ent = "datosemae/bronze/Entidades"
container_name_paa = "datosemae/bronze/PAA"

### Guardamos los archivos que dejamos en la carpeta de Large Data
ruta_LD='/dropbox/Dropbox/CCE-projects/Large Data'


### Cargamos los archivos parquet en el contenedor

# Cargamos el archivo de entidades
ruta_ent_small=ruta_LD+'/Entidades/Sat_Company.parquet'
ruta_ent_full=ruta_LD+'/Entidades/Sat_Company_full.parquet'

blob_client_ent_small = blob_service_client.get_blob_client(container=container_name_ent, blob='Sat_Company.parquet')
blob_client_ent_full = blob_service_client.get_blob_client(container=container_name_ent, blob='Sat_Company_full.parquet')

with open(ruta_ent_small, "rb") as data:
    blob_client_ent_small.upload_blob(data, overwrite=True)

with open(ruta_ent_full, "rb") as data:
    blob_client_ent_full.upload_blob(data, overwrite=True)

In [19]:
# Referencia al contenedor donde se guardará el archivo
container_name_ent = "datosemae/bronze/Entidades"

### Guardamos los archivos que dejamos en la carpeta de Large Data
ruta_LD='/dropbox/Dropbox/CCE-projects/Large Data'


### Cargamos los archivos parquet en el contenedor

# Cargamos el archivo de entidades
ruta_ent_small=ruta_LD+'/Entidades/Entidades_SECOP_II.parquet'
ruta_ent_full=ruta_LD+'/Entidades/Entidades_SECOP_II_FULL.parquet'

blob_client_ent_small = blob_service_client.get_blob_client(container=container_name_ent, blob='Entidades_SECOP_II.parquet')
blob_client_ent_full = blob_service_client.get_blob_client(container=container_name_ent, blob='Entidades_SECOP_II_FULL.parquet')

with open(ruta_ent_small, "rb") as data:
    blob_client_ent_small.upload_blob(data, overwrite=True)

with open(ruta_ent_full, "rb") as data:
    blob_client_ent_full.upload_blob(data, overwrite=True)

## Lectura de datos desde DataLake

Para leer los datos desde el datalake, se debe ejecutar la siguiente consulta:


In [1]:
import pandas as pd
import pyarrow.parquet as pq
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import tempfile
# URL del punto final del servicio Blob y token SAS
url_blob = "https://adlindicadoresemae.blob.core.windows.net/"
token_sas = "sp=racwdlmeop&st=2023-06-09T05:36:44Z&se=2023-06-09T13:36:44Z&spr=https&sv=2022-11-02&sr=c&sig=FE5BaP6bfBsXzBrcJGyobQJ92RuRgbwrZCMWeKrsAd8%3D"


### Cargamos los archivos parquet del contenedor
# Conexión a la cuenta de almacenamiento de Azure utilizando el token SAS
connect_str = f"BlobEndpoint={url_blob};SharedAccessSignature={token_sas}"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Referencia al contenedor donde se encuentran los archivos
container_name_ent = "datosemae/bronze/Entidades"
container_client_ent = blob_service_client.get_container_client(container_name_ent)

# Nombre del archivo a leer
blob_name_ent_small = "Entidades_SECOP_II.parquet"
blob_name_ent_full = "Entidades_SECOP_II_FULL.parquet"

In [2]:
# Descarga del archivo Sat_Company.parquet desde Azure Blob Storage
blob_client_ent_small = container_client_ent.get_blob_client(blob_name_ent_small)
download_stream_ent_small = blob_client_ent_small.download_blob()
downloaded_blob_ent_small = download_stream_ent_small.readall()
# Escritura del archivo descargado en un archivo temporal
with tempfile.TemporaryFile() as tmp_file:
    tmp_file.write(downloaded_blob_ent_small)

    # Conversión del archivo Parquet a un DataFrame de Pandas
    df_entidades = pd.read_parquet(tmp_file)

df_entidades.shape

(20045, 8)

In [3]:
# Descarga del archivo Sat_Company_full.parquet desde Azure Blob Storage
blob_client_ent_full = container_client_ent.get_blob_client(blob_name_ent_full)
download_stream_ent_full = blob_client_ent_full.download_blob()
downloaded_blob_ent_full = download_stream_ent_full.readall()
# Escritura del archivo descargado en un archivo temporal
with tempfile.TemporaryFile() as tmp_file:
    tmp_file.write(downloaded_blob_ent_full)

    # Conversión del archivo Parquet a un DataFrame de Pandas
    DF_ENTIDADES = pd.read_parquet(tmp_file,engine='fastparquet')
# Visualización del DataFrame Sat_Company_full.parquet
print(DF_ENTIDADES.shape)

(27604, 138)


In [4]:
df_entidades

,Codigo,Nombre Legal,NIT,Entidad,Creacion_Fecha,Count Nombre Legal,Count NIT,Count Entidad
0,700000011,VORTAL COMPRADOR (ENTIDAD DE SISTEMA),800211401,True,2015-02-14 11:06:02.614891,1,1,1
8,700003015,VORTAL OPERATIONS,000000001,True,2015-02-16 09:34:20.870667,1,1,1
42,700007024,FISCALIA SUBDIRECCION NOR OCCIDENTAL,800187568,True,2015-02-18 08:29:29.042615,1,1,1
184,700028038,ALCALDIA MUNICIPIO DE VILLANUEVA BOLIVAR,890481192,True,2015-03-05 16:27:37.969787,1,1,1
190,700028095,FUNDACIÓN DE FOMENTO TURÍSTICO DEPORTIVO Y CUL...,9002109871,True,2015-03-05 21:58:40.591303,1,1,1
...,...,...,...,...,...,...,...,...
1128324,721867455,EMPRESA DE DESARROLLO URBANO DEL PACIFICO,901653745,True,2023-05-10 20:07:38.732332,1,1,1
1128340,721867612,INSTITUCION EDUCATIVA DISTRITAL EL PARAISO,802007590,True,2023-05-10 20:29:14.261666,1,1,1
1128560,721869816,INSTITUCION EDUCASTIVA LAS AREPAS,9002272671,True,2023-05-10 21:46:34.131316,1,1,1
1128849,721873677,LAURA MARCELA CABRA,1007296094,True,2023-05-11 04:37:47.697546,1,1,1


### Creador de código HASH

In [25]:
dm.generate_hash_key('0')

'5FECEB66FFC86F38D952786C6D696C79C2DBC239DD4E91B46729D73A27FB57E9'

## Carga Versión Previa de MDM

Cargamos la base desde la ruta `../../../Data/Raw/`

In [26]:
MDM_VP=pd.read_excel(ruta_DATA+'/20220714_MDM_Entidades_v6.xlsx',sheet_name='MDM')
MDM_VP.columns
MDM_VP['HashKey']=[dm.generate_hash_key(str(i+7456788)) for i in range(1,len(MDM_VP)+1)]
MDM_VP['NIT_Secop']=MDM_VP['NIT_Secop'].fillna(-1).astype(int).astype(str)

In [27]:
# Se cambia el tipo de variables Codigo a str

df_entidades['Codigo']=df_entidades['Codigo'].astype(str)
MDM_VP['codigo_entidad']=MDM_VP['codigo_entidad'].fillna(-1).astype(int).astype(str)

In [28]:
## Integramos a traves de código de entidad a MDM_VP y df_entidades

DF_Integrado=pd.merge(df_entidades,MDM_VP,how='inner',left_on='Codigo',right_on='codigo_entidad')
DF_Integrado.shape

(5249, 32)

In [29]:
# Fuentes que contengan SECOP I Y SECOP II

MDM_SI_SII=MDM_VP[MDM_VP['Fuente'].isin(['SECOP II','SECOP I'])]
MDM_SI_SII=MDM_SI_SII[MDM_SI_SII['codigo_sigep']!=0]
#MDM_SI_SII

In [30]:
# Se crea la variable Llave_NIT, con Nit SECOP más codigo SIGEP

MDM_SI_SII['Llave_NIT']=(MDM_SI_SII['NIT_Secop'].astype(str)+'-'+MDM_SI_SII['codigo_sigep'].astype(str)).astype(str)
MDM_SI_SII['Llave_NIT'].value_counts()[MDM_SI_SII['Llave_NIT'].value_counts()<=1]

Llave_NIT
890503106-222     1
800188421-4747    1
0-4745            1
891780042-212     1
890980998-217     1
                 ..
890984265-2625    1
890984030-2624    1
890980964-2623    1
800097180-2622    1
900160887-2867    1
Name: count, Length: 5100, dtype: int64

In [31]:
# Se crea la variable Llave_NIT, con Nit SECOP más codigo SIGEP más fuente

MDM_SI_SII['Llave_NIT']=(MDM_SI_SII['NIT_Secop'].astype(str)+'-'+MDM_SI_SII['codigo_sigep'].astype(str)+'-'+MDM_SI_SII['Fuente']).astype(str)
LLAVES=MDM_SI_SII['Llave_NIT'].value_counts()[MDM_SI_SII['Llave_NIT'].value_counts()<=1].index.tolist()

In [32]:
len(LLAVES)

7925

In [33]:
MDM_SI_SII['Fuente'].value_counts()

Fuente
SECOP I     5239
SECOP II    3171
Name: count, dtype: int64

In [34]:
# Lista de llaves limpias

LLAVES_LIMPIAS=[i.replace('-SECOP II','').replace('-SECOP I','') for i in LLAVES]
len(LLAVES_LIMPIAS)

7925

In [35]:
MDM_SI_SII.columns

Index(['codigo_entidad', 'codigo_sigep', 'NIT_Secop', 'N_Entidad_Final',
       'N_Entidad_Plataforma', 'Nombre Estandar', 'N_Entidad_FP',
       'codigo_sigep.1', 'Fuente', 'orden_Entidad', 'rama_Entidad',
       'sector_Entidad', 'llave', 'Id_Ciudad', 'Municipio', 'NOMBRE_DPT',
       'Obligada SECOP II', 'Entidad Ciudad Capital', 'TVEC',
       'validacion_nombre', 'Año_Obligatoriedad', 'Mes_Ingreso',
       'Trimestre_Ingreso', 'HashKey', 'Llave_NIT'],
      dtype='object')

In [36]:
# Limpieza de dataframe MDM_SI_SII

MDM_SI_SII['Llave_Limpia']=(MDM_SI_SII['NIT_Secop'].astype(str)+'-'+MDM_SI_SII['codigo_sigep'].astype(str)).astype(str)
Entidades=[]
for i,llave in enumerate(LLAVES_LIMPIAS):
    Info=[]
    DF=MDM_SI_SII[MDM_SI_SII['Llave_Limpia']==llave]

    if DF.shape[0]<=2:
        Info.append(DF['NIT_Secop'].unique().tolist()[0])
        Info.append(DF['codigo_sigep'].unique().tolist()[0])
        Info.append(DF['N_Entidad_Plataforma'].unique().tolist()[0])
        Info.append(DF['orden_Entidad'].unique().tolist()[0])
        Info.append(DF['rama_Entidad'].unique().tolist()[0])
        Info.append(DF['sector_Entidad'].unique().tolist()[0])
        Info.append(DF['Obligada SECOP II'].unique().tolist()[0])
        Info.append(DF['Entidad Ciudad Capital'].unique().tolist()[0])
        Info.append(DF['Año_Obligatoriedad'].unique().tolist()[0])
        if DF[DF['Fuente']=='SECOP II'].shape[0]>0:
            Info.append(DF[DF['Fuente']=='SECOP II']['codigo_entidad'].unique().tolist()[0])
        else:
            Info.append('No registra')
        
        if DF[DF['Fuente']=='SECOP I'].shape[0]>0:
            Info.append(DF[DF['Fuente']=='SECOP I']['codigo_entidad'].unique().tolist()[0])
        else:
            Info.append('No registra')

        Entidades.append(Info)

MDM_limpio=pd.DataFrame(Entidades,columns=['NIT_Secop','codigo_sigep','N_Entidad_Plataforma','orden_Entidad','rama_Entidad','sector_Entidad','Obligada SECOP II','Entidad Ciudad Capital','Año_Obligatoriedad','codigo_entidad_SECOP II','codigo_entidad_SECOP I'])
MDM_limpio.shape

(7878, 11)

In [37]:
# Se descarta Nit_Secop igual a cero

MDM_limpio=MDM_limpio[MDM_limpio['NIT_Secop']!='0']

In [38]:
# Se eliminan duplicados

MDM_limpio.drop_duplicates(keep='first',inplace=True)
MDM_limpio.shape

(6215, 11)

In [39]:
# Se define llave NIT

MDM_VP['Llave_NIT']=(MDM_VP['NIT_Secop'].astype(str)+'-'+MDM_VP['codigo_sigep'].astype(str)).astype(str)
MDM_limpio['Llave_NIT']=(MDM_limpio['NIT_Secop'].astype(str)+'-'+MDM_limpio['codigo_sigep'].astype(str)).astype(str)
MDM_VP_R=MDM_VP[~MDM_VP['Llave_NIT'].isin(MDM_limpio['Llave_NIT'])]
MDM_VP_R.shape

(11587, 25)

In [40]:
# Se descarta fuente TVEC

MDM_VP_R=MDM_VP_R[MDM_VP_R['Fuente']!='TVEC']

In [41]:
# Limpieza de dataframe MDM_VP_R

MDM_VP_R['Llave_Nombre']=MDM_VP_R['Llave_NIT']
Llave_Nombre=MDM_VP_R['Llave_Nombre'].value_counts()[MDM_VP_R['Llave_Nombre'].value_counts()==2].index.tolist()
Entidades=[]
for llave in Llave_Nombre:
    DF=MDM_VP_R[MDM_VP_R['Llave_Nombre']==llave]
    Info=[]
    if DF['Fuente'].nunique()==2:
        Info.append(DF['NIT_Secop'].unique().tolist()[0])
        Info.append(DF['codigo_sigep'].unique().tolist()[0])
        Info.append(DF['N_Entidad_Plataforma'].unique().tolist()[0])
        Info.append(DF['orden_Entidad'].unique().tolist()[0])
        Info.append(DF['rama_Entidad'].unique().tolist()[0])
        Info.append(DF['sector_Entidad'].unique().tolist()[0])
        Info.append(DF['Obligada SECOP II'].unique().tolist()[0])
        Info.append(DF['Entidad Ciudad Capital'].unique().tolist()[0])
        Info.append(DF['Año_Obligatoriedad'].unique().tolist()[0])
        if DF[DF['Fuente']=='SECOP II'].shape[0]>0:
            Info.append(DF[DF['Fuente']=='SECOP II']['codigo_entidad'].unique().tolist()[0])
        else:
            Info.append('No registra')
        
        if DF[DF['Fuente']=='SECOP I'].shape[0]>0:
            Info.append(DF[DF['Fuente']=='SECOP I']['codigo_entidad'].unique().tolist()[0])
        else:
            Info.append('No registra')

        Entidades.append(Info)

MDM_limpio2=pd.DataFrame(Entidades,columns=['NIT_Secop','codigo_sigep','N_Entidad_Plataforma','orden_Entidad','rama_Entidad','sector_Entidad','Obligada SECOP II','Entidad Ciudad Capital','Año_Obligatoriedad','codigo_entidad_SECOP II','codigo_entidad_SECOP I'])
MDM_limpio2.shape        

(1035, 11)

In [42]:
# Se descarta Nit_secop igual a cero y se eliminan duplicados

MDM_limpio2=MDM_limpio2[MDM_limpio2['NIT_Secop']!='0']
MDM_limpio2.drop_duplicates(keep='first',inplace=True)
MDM_limpio2.shape

(1035, 11)

In [43]:
# Se concatenan los dos dataframes limpios MDM

MDM_limpio_Total=pd.concat([MDM_limpio,MDM_limpio2],axis=0)
MDM_limpio_Total.drop_duplicates(keep='first',inplace=True)
MDM_limpio_Total.shape

(7250, 12)

In [44]:
# Se crea la variable llave_NIT

MDM_limpio_Total['Llave_NIT']=(MDM_limpio_Total['NIT_Secop'].astype(str)+'-'+MDM_limpio_Total['codigo_sigep'].astype(str)).astype(str)

Hay cruce con la versión previa del MDM con 5249 entidades

In [45]:
MDM_VP_R=MDM_VP[~MDM_VP['Llave_NIT'].isin(MDM_limpio_Total['Llave_NIT'])]
MDM_VP_R=MDM_VP_R[MDM_VP_R['Fuente']!='TVEC']
MDM_VP_R.shape

(7250, 25)

In [46]:
## Ingresamos las entidades con NIT único

LLAVES_NIT_UNICO=MDM_VP_R['NIT_Secop'].value_counts()[MDM_VP_R['NIT_Secop'].value_counts()==1].index.tolist()
Entidades=[]
for llave in LLAVES_NIT_UNICO:
    DF=MDM_VP_R[MDM_VP_R['NIT_Secop']==llave]
    Info=[]
    if DF['Fuente'].nunique()==1:
        Info.append(DF['NIT_Secop'].unique().tolist()[0])
        Info.append(DF['codigo_sigep'].unique().tolist()[0])
        Info.append(DF['N_Entidad_Plataforma'].unique().tolist()[0])
        Info.append(DF['orden_Entidad'].unique().tolist()[0])
        Info.append(DF['rama_Entidad'].unique().tolist()[0])
        Info.append(DF['sector_Entidad'].unique().tolist()[0])
        Info.append(DF['Obligada SECOP II'].unique().tolist()[0])
        Info.append(DF['Entidad Ciudad Capital'].unique().tolist()[0])
        Info.append(DF['Año_Obligatoriedad'].unique().tolist()[0])
        if DF[DF['Fuente']=='SECOP II'].shape[0]>0:
            Info.append(DF[DF['Fuente']=='SECOP II']['codigo_entidad'].unique().tolist()[0])
        else:
            Info.append('No registra')
        if DF[DF['Fuente']=='SECOP I'].shape[0]>0:
            Info.append(DF[DF['Fuente']=='SECOP I']['codigo_entidad'].unique().tolist()[0])
        else:
            Info.append('No registra')

        Entidades.append(Info)


MDM_limpio3=pd.DataFrame(Entidades,columns=['NIT_Secop','codigo_sigep','N_Entidad_Plataforma','orden_Entidad','rama_Entidad','sector_Entidad','Obligada SECOP II','Entidad Ciudad Capital','Año_Obligatoriedad','codigo_entidad_SECOP II','codigo_entidad_SECOP I'])
MDM_limpio3.shape    

(5009, 11)

In [47]:
# Se realiza el merge con el daframe limpio y con la llev de NIT_secop

Merge_Limpio3_df=pd.merge(MDM_limpio3,df_entidades,how='left',left_on='NIT_Secop',right_on='NIT')
Merge_Limpio3_df.columns

Index(['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
       'rama_Entidad', 'sector_Entidad', 'Obligada SECOP II',
       'Entidad Ciudad Capital', 'Año_Obligatoriedad',
       'codigo_entidad_SECOP II', 'codigo_entidad_SECOP I', 'Codigo',
       'Nombre Legal', 'NIT', 'Entidad', 'Creacion_Fecha',
       'Count Nombre Legal', 'Count NIT', 'Count Entidad'],
      dtype='object')

In [48]:
# Se definen variables

Merge_Limpio3_df['codigo_entidad_SECOP II']=Merge_Limpio3_df['Codigo']
Merge_Limpio3_df['N_Entidad_Plataforma']=Merge_Limpio3_df['Nombre Legal']
Merge_Limpio3_df=Merge_Limpio3_df[['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
       'rama_Entidad', 'sector_Entidad', 'Obligada SECOP II',
       'Entidad Ciudad Capital', 'Año_Obligatoriedad',
       'codigo_entidad_SECOP II', 'codigo_entidad_SECOP I']]

# Se concatenan dataframes y se eliminan duplicados
MDM_limpio_Total=pd.concat([MDM_limpio_Total,Merge_Limpio3_df],axis=0)
MDM_limpio_Total.drop_duplicates(keep='first',inplace=True)
MDM_limpio_Total.shape

(12878, 12)

In [49]:
# Se filtra por codigo entidad Secop II igual a no registra y se eliminan duplicados

MDM_INT_df=MDM_limpio_Total[MDM_limpio_Total['codigo_entidad_SECOP II']=='No registra']
print(MDM_INT_df.shape)
MDM_INT_df_2=pd.merge(MDM_INT_df,df_entidades,how='left',left_on='NIT_Secop',right_on='NIT')
MDM_INT_df_2=MDM_INT_df_2[~MDM_INT_df_2['Codigo'].duplicated()]

(3419, 12)


In [50]:
MDM_INT_df_2.columns

Index(['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
       'rama_Entidad', 'sector_Entidad', 'Obligada SECOP II',
       'Entidad Ciudad Capital', 'Año_Obligatoriedad',
       'codigo_entidad_SECOP II', 'codigo_entidad_SECOP I', 'Llave_NIT',
       'Codigo', 'Nombre Legal', 'NIT', 'Entidad', 'Creacion_Fecha',
       'Count Nombre Legal', 'Count NIT', 'Count Entidad'],
      dtype='object')

In [51]:
# Se seleccionan variables de interés, se concantena y se eliminan duplicados

MDM_Limpio4=MDM_INT_df_2[['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
       'rama_Entidad', 'sector_Entidad', 'Obligada SECOP II',
       'Entidad Ciudad Capital', 'Año_Obligatoriedad',
       'codigo_entidad_SECOP II', 'codigo_entidad_SECOP I']]

MDM_limpio_Total=pd.concat([MDM_limpio_Total,MDM_Limpio4],axis=0)
MDM_limpio_Total.drop_duplicates(keep='first',inplace=True)
MDM_limpio_Total.shape

(14626, 12)

In [52]:
# Se define variable Llave_NIT

MDM_limpio_Total['Llave_NIT']=(MDM_limpio_Total['NIT_Secop'].astype(str)+'-'+MDM_limpio_Total['codigo_sigep'].astype(str)).astype(str)

MDM_VP_R=MDM_VP[~MDM_VP['Llave_NIT'].isin(MDM_limpio_Total['Llave_NIT'])]
MDM_VP_R=MDM_VP_R[MDM_VP_R['Fuente']!='TVEC']
MDM_VP_R.shape

(2241, 25)

In [53]:
MDM_INT_4=pd.merge(MDM_VP_R,df_entidades,how='left',left_on='codigo_entidad',right_on='Codigo')
Entidades=[]

cod_SI=[]
for ind in MDM_INT_4.index:
    if MDM_INT_4.loc[ind,'Fuente']=='SECOP II':
        MDM_INT_4.loc[ind,'codigo_entidad']='No Registra'

MDM_INT_4=MDM_INT_4[['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
           'rama_Entidad', 'sector_Entidad', 'Obligada SECOP II',
           'Entidad Ciudad Capital', 'Año_Obligatoriedad',
           'Codigo', 'codigo_entidad','Llave_NIT']]
MDM_INT_4.columns=['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
              'rama_Entidad', 'sector_Entidad', 'Obligada SECOP II',
                'Entidad Ciudad Capital', 'Año_Obligatoriedad', 
                'codigo_entidad_SECOP II', 'codigo_entidad_SECOP I','Llave_NIT']

In [54]:
MDM_limpio_Total=pd.concat([MDM_limpio_Total,MDM_INT_4],axis=0)
MDM_limpio_Total.drop_duplicates(keep='first',inplace=True)
MDM_limpio_Total.shape

(15119, 12)

In [55]:
MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP II']

2240    703971531
2240    717943054
2240          NaN
Name: codigo_entidad_SECOP II, dtype: object

In [56]:
## Generamos Código de Entidad

MDM_limpio_Total.reset_index(inplace=True,drop=True)
MDM_limpio_Total['codigo_entidad_SECOP II'].fillna('No Registra',inplace=True)
MDM_limpio_Total['codigo_entidad_SECOP I'].fillna('No Registra',inplace=True)
MDM_limpio_Total['N_Entidad_Plataforma'].fillna('No Registra',inplace=True)
MDM_limpio_Total['drop_file']=MDM_limpio_Total['codigo_entidad_SECOP II'].astype(str).str.lower()+MDM_limpio_Total['codigo_entidad_SECOP I'].astype(str).str.lower()
MDM_limpio_Total=MDM_limpio_Total[MDM_limpio_Total['drop_file']!='no registrano registra']
for ind in MDM_limpio_Total.index:
    if MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP II']=='No Registra':
        cod=str(MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP I'])+'S1'
    elif MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP I']=='No Registra':
        cod=str(MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP II'])+'S2'
    else:
        cod=MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP II']+MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP I']+'S12'
    cod+='-'+MDM_limpio_Total.loc[ind,'N_Entidad_Plataforma']
    MDM_limpio_Total.loc[ind,'codigo_entidad']=dm.generate_hash_key(cod)

MDM_limpio_Total['codigo_entidad']=MDM_limpio_Total['codigo_entidad'].astype(str)
MDM_limpio_Total = dm.eliminar_duplicados(MDM_limpio_Total,'codigo_entidad')
MDM_limpio_Total['codigo_entidad'].is_unique

True

In [57]:
# Se filtra donde Codigo de entidades sea igual a los codigos de entidad de secop II 

df_entidades_huer=df_entidades[~df_entidades.Codigo.isin(MDM_limpio_Total['codigo_entidad_SECOP II'].unique().tolist())]
df_entidades_huer.shape

(11304, 8)

In [58]:
# Diccionarios de orden, rama y sector

Diccionario_orden={'Territorial':'Territorial','National':'Nacional','Autonomous': 'Corporación Autónoma'}
Diccionario_rama={'Executive':'Ejecutivo','Legislative':'Legislativo','Judicial':'Judicial','Autonomous':'Corporación Autónoma'}
Diccionario_sector={'NationalEducation':'Educación Nacional',
                    'PublicService':'Servicio Público',
                    'HealthSocialProtect':'Salud y Protección Social',
                    'Defense':'Defensa',
                    'Labour':'Trabajo',
                    'JusticeLaw':'Ley de Justicia',
                    'Sports':'deportes',
                    'Environment':'Ambiente y Desarrollo Sostenible',
                    'Culture':'Cultura',
                    'Agriculture':'agricultura',
                    'Territory':'Vivienda, Ciudad y Territorio',
                    'Transport':'Transporte',
                    'Planning':'Planeación',
                    'Industry':'Industria',
                    'FinancePublicCredit':'Hacienda y Crédito Público',
                    'MinesEnergy':'Minas y Energía',
                    'SocialInclusion':'Inclusión Social y Reconciliación',
                    'ScienceTech':'Ciencia y Tecnología',
                    'TI':'Tecnologías de la Información y las Comunicaciones',
                    'Foreign':'Relaciones Exteriores',
                    'Interior':'interior',
                    'StatisticalInfo':'Información Estadística',
                    'PresidencyRep':'Presidencia de la República',
                    'Intelligence':'Inteligencia Estratégica y Contrainteligencia'}

In [59]:
# Se eliminan duplicados y se agrupan por código

INT_ENT=DF_ENTIDADES[['Code','CompanySectorCode','CompanyBranchCode','ScopeOfInfluenceCode']].drop_duplicates().reset_index(drop=True)
INT_ENT=INT_ENT.groupby('Code').agg({'CompanySectorCode':'first','CompanyBranchCode':'first','ScopeOfInfluenceCode':'first'}).reset_index()

In [60]:
MDM_limpio_Total.columns

Index(['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
       'rama_Entidad', 'sector_Entidad', 'Obligada SECOP II',
       'Entidad Ciudad Capital', 'Año_Obligatoriedad',
       'codigo_entidad_SECOP II', 'codigo_entidad_SECOP I', 'Llave_NIT',
       'drop_file', 'codigo_entidad'],
      dtype='object')

In [61]:
# Se definen variables

df_entidades_huer['codigo_entidad_SECOP II']=df_entidades_huer['Codigo'].astype(str)
df_entidades_huer['codigo_entidad_SECOP I']='No Registra'
df_entidades_huer['codigo_sigep']='No Registra'
df_entidades_huer['NIT_Secop']=df_entidades_huer['NIT']
df_entidades_huer['N_Entidad_Plataforma']=df_entidades_huer['Nombre Legal']

In [62]:
# Se realiza el merge por código

df_entidades_huer=pd.merge(df_entidades_huer,INT_ENT,how='left',left_on='Codigo',right_on='Code')
df_entidades_huer['orden_Entidad']=df_entidades_huer['CompanySectorCode'].map(Diccionario_orden)
df_entidades_huer['rama_Entidad']=df_entidades_huer['CompanyBranchCode'].map(Diccionario_rama)
df_entidades_huer['sector_Entidad']=df_entidades_huer['ScopeOfInfluenceCode'].map(Diccionario_sector)


### Para las tres anteriores llenamos vacios con 'No Definido'

df_entidades_huer['orden_Entidad']=df_entidades_huer['orden_Entidad'].fillna('No Definido')
df_entidades_huer['rama_Entidad']=df_entidades_huer['rama_Entidad'].fillna('No Definido')
df_entidades_huer['sector_Entidad']=df_entidades_huer['sector_Entidad'].fillna('No Definido')


In [63]:
# Se concatena MDM_limpio_total con dataframe de entidades huer

MDM_limpio_Total=pd.concat([MDM_limpio_Total,df_entidades_huer],axis=0)
MDM_limpio_Total.reset_index(inplace=True,drop=True)

In [64]:
## Generamos Código de Entidad

MDM_limpio_Total.reset_index(inplace=True,drop=True)
MDM_limpio_Total['codigo_entidad_SECOP II'].fillna('No Registra',inplace=True)
MDM_limpio_Total['codigo_entidad_SECOP I'].fillna('No Registra',inplace=True)
MDM_limpio_Total['N_Entidad_Plataforma'].fillna('No Registra',inplace=True)
MDM_limpio_Total['drop_file']=MDM_limpio_Total['codigo_entidad_SECOP II'].astype(str).str.lower()+MDM_limpio_Total['codigo_entidad_SECOP I'].astype(str).str.lower()
MDM_limpio_Total=MDM_limpio_Total[MDM_limpio_Total['drop_file']!='no registrano registra']
for ind in MDM_limpio_Total.index:
    if MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP II']=='No Registra':
        cod=str(MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP I'])+'S1'
    elif MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP I']=='No Registra':
        cod=str(MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP II'])+'S2'
    else:
        cod=MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP II']+MDM_limpio_Total.loc[ind,'codigo_entidad_SECOP I']+'S12'
    cod+='-'+MDM_limpio_Total.loc[ind,'N_Entidad_Plataforma']
    MDM_limpio_Total.loc[ind,'codigo_entidad']=dm.generate_hash_key(cod)

MDM_limpio_Total['codigo_entidad']=MDM_limpio_Total['codigo_entidad'].astype(str)
MDM_limpio_Total.drop_duplicates(subset=['codigo_entidad'],keep='first',inplace=True)
MDM_limpio_Total['codigo_entidad'].is_unique

True

In [65]:
# Se seleccionan variables de interés

MDM_limpio_Total=MDM_limpio_Total[['NIT_Secop', 'codigo_sigep', 'N_Entidad_Plataforma', 'orden_Entidad',
       'rama_Entidad', 'sector_Entidad', 'codigo_entidad_SECOP II', 'codigo_entidad_SECOP I',
       'codigo_entidad']]

In [66]:
MDM_limpio_Total

,NIT_Secop,codigo_sigep,N_Entidad_Plataforma,orden_Entidad,rama_Entidad,sector_Entidad,codigo_entidad_SECOP II,codigo_entidad_SECOP I,codigo_entidad
0,820000265,4320,BOYACA - FONVICHIQ - FONDO DE VIVIENDA DE INTE...,Territorial,NaN,No aplica,No registra,288968160,61F38D49A7C217EE15D27B33DC9C43E1A49E6F0A7A7DF7...
1,900128208,4182,ATLANTICO - EDUMAS - EMPRESA DE DESARROLLO URB...,Territorial,NaN,No aplica,705351526,288399170,FCB07C4B42AAE75FFF2440D09DCA0F6F2B434942E96A15...
2,800165392,4320,FONDO DE VIVIENDA DE INTERES SOCIAL Y REFORMA ...,Territorial,Ejecutivo,"Sector de Vivienda, Ciudad y Territorio",704717206,No registra,6AA77ED7E02F1DE3975AC70A0A9C25B0FD782EFF9899FE...
3,900094866,4320,HUILA - FONDO DE VIVIENDA DE INTERES SOCIAL,Territorial,NaN,No aplica,705008498,24100117,5FD81A94C7593C7FB3FDA6B3A8FBCA8DD9AC87DC291291...
4,900262356,4205,ANTIOQUIA - EMPRESA DE SERVICIOS PUBLICOS DOMI...,Territorial,NaN,Sector de Justicia y del Derecho,705402667,20562812,3E75C7E19354EFA2FF8D694213AE3BE3D50F88C9754D5B...
...,...,...,...,...,...,...,...,...,...
26358,830115297,No Registra,MARCIA CAROLINA PORRAS,No Definido,Ejecutivo,No Definido,721867224,No Registra,97E437C02ACAE4CE3DE6AC1FFF079D650C69A5415B80EA...
26359,901653745,No Registra,EMPRESA DE DESARROLLO URBANO DEL PACIFICO,No Definido,Ejecutivo,No Definido,721867455,No Registra,06981563AB177A2CEB795CFBF93ABE41C9E49626D42FD6...
26360,9002272671,No Registra,INSTITUCION EDUCASTIVA LAS AREPAS,No Definido,Legislativo,No Definido,721869816,No Registra,F63D80251239A222A4FB5201C690E946419268867C2B43...
26361,1007296094,No Registra,LAURA MARCELA CABRA,No Definido,Ejecutivo,No Definido,721873677,No Registra,8060CC65ED06042500C8A92CA7ECEB12F99BBC7951583F...


In [67]:
# Se eliminan duplicados en codigo entidad SECOP I Y II

MDM_SII = dm.eliminar_duplicados(MDM_limpio_Total,'codigo_entidad_SECOP II')
MDM_SI = dm.eliminar_duplicados(MDM_limpio_Total,'codigo_entidad_SECOP I')

# Se concatenan SECOP I y SECOP II, se resetea index y se eliminan duplicados

MDM_Total=pd.concat([MDM_SI,MDM_SII],axis=0)
MDM_Total.reset_index(inplace=True,drop=True)
MDM_Total.drop_duplicates(keep='first',inplace=True)
MDM_Total.shape

(26337, 9)

In [68]:
# Se definen nombres de columnas

MDM_limpio_Total.columns=['NIT_SECOP', 'Codigo_SIGEP', 'ID_Entidad_Plataforma', 'Orden_Entidad',
'Rama_Entidad', 'Sector_Entidad', 'Codigo_Entidad_SECOP_II', 'Codigo_Entidad_SECOP_I',
'Codigo_Entidad']
MDM_SII.columns=MDM_limpio_Total.columns
MDM_SI.columns=MDM_limpio_Total.columns
MDM_Total.columns=MDM_limpio_Total.columns

In [69]:
### Guardamos el archivo

MDM_limpio_Total.to_excel('../../../Data/Processed/MDM_limpio_Total.xlsx',index=False)

MDM_SII.to_excel('../../../Data/Processed/MDM_SII.xlsx',index=False)
MDM_SI.to_excel('../../../Data/Processed/MDM_SI.xlsx',index=False)
MDM_Total.to_excel('../../../Data/Processed/MDM_Total.xlsx',index=False)